# MLLib trial

In this notebook will trial MLLib using a public dataset from Big Query.
Have taken a dataset covering taxi journeys in New York. Have used Big Query to perform some basic cleaning and feature engineering so the data is a fine state to model.
In this notebook will model the data using the spark library mllib.

Full dataset is 1B rows but for this trial will use. sample dataset of 1000 rows.


# Load libraries and create spark session

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import isnan, when, count, col

In [2]:
spark = SparkSession.builder.master('yarn').appName('spark-test').getOrCreate()

# Load data from Big Query

Big Query connector loaded into cluster enviroment on initilisation. 
https://cloud.google.com/dataproc/docs/concepts/connectors/bigquery

In [3]:
# Use the Cloud Storage bucket for temporary BigQuery export data used
# by the connector.
bucket = "pyspark_temp_20201006_eu"
spark.conf.set('temporaryGcsBucket', bucket)

In [24]:
# Load data from BigQuery.
taxi= spark.read.format('bigquery').option('table', 'sap-ds-demo:big_data_demo_ew2.nyc_taxi_1000').load()
taxi.createOrReplaceTempView('taxi')

# Check data

In [14]:
%%timeit
taxi.count()

89 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%time
taxi.count()

CPU times: user 2.1 ms, sys: 278 µs, total: 2.38 ms
Wall time: 67.8 ms


1000

In [6]:
taxi.printSchema()

root
 |-- vendor_id: string (nullable = true)
 |-- journey_length: long (nullable = true)
 |-- year: long (nullable = true)
 |-- month: long (nullable = true)
 |-- dayofweek: long (nullable = true)
 |-- pu_hour: long (nullable = true)
 |-- do_hour: long (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- rate_code: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pu_m_center: double (nullable = true)
 |-- do_m_center: double (nullable = true)
 |-- pu_m_jfk: double (nullable = true)
 |-- do_m_jfk: double (nullable = true)



In [7]:
taxi.limit(10).toPandas()

,vendor_id,journey_length,year,month,dayofweek,pu_hour,do_hour,passenger_count,trip_distance,pickup_longitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,total_amount,pu_m_center,do_m_center,pu_m_jfk,do_m_jfk
0,CMT,0,2009,9,1,0,0,1,0.0,-74.006727,40.712822,0,-74.006624,40.713165,No Charge,2.5,6340.676613,6320.606246,6340.676613,6320.606246
1,VTS,1,2009,11,1,0,0,1,0.0,-74.002962,40.727873,,-74.003005,40.727885,CASH,3.5,5714.454928,5718.001754,5714.454928,5718.001754
2,VTS,1,2009,11,1,0,0,1,0.0,-73.999203,40.734107,,-73.999203,40.734107,Credit,5.5,5403.354587,5403.354587,5403.354587,5403.354587
3,CMT,0,2009,7,7,0,0,1,0.0,-73.992394,40.724811,0,-73.992392,40.724819,Cash,2.5,4858.932353,4858.647049,4858.932353,4858.647049
4,CMT,0,2009,11,5,0,0,1,0.0,-73.986512,40.761462,0,-73.986529,40.761480,Cash,3.5,5515.768986,5518.134853,5515.768986,5518.134853
5,VTS,-4,2009,3,6,0,0,5,0.0,-73.995103,40.731277,,-74.008030,40.730933,CASH,3.0,5044.531882,6133.357940,5044.531882,6133.357940
6,CMT,0,2009,7,6,0,0,1,0.0,-73.984971,40.767525,0,-73.984971,40.767525,Cash,2.5,5864.957050,5864.957050,5864.957050,5864.957050
7,CMT,0,2009,7,4,0,0,1,0.0,-74.011516,40.714566,0,-74.011075,40.714169,Cash,2.5,6670.761685,6646.982691,6670.761685,6646.982691
8,DDS,0,2009,5,2,0,0,1,0.0,-73.982345,40.767384,,-73.981823,40.767653,CASH,3.0,5697.795231,5688.875095,5697.795231,5688.875095
9,DDS,0,2009,5,6,0,0,1,0.0,-73.962730,40.760227,,-73.960835,40.759509,CASH,3.0,4025.912499,3869.711833,4025.912499,3869.711833


In [28]:
%%time
taxi.groupBy("year").count().sort("year",ascending=False).toPandas()

CPU times: user 16.2 ms, sys: 469 µs, total: 16.6 ms
Wall time: 1.73 s


,year,count
0,2016,1
1,2014,172
2,2013,201
3,2012,158
4,2011,135
5,2010,151
6,2009,182


In [29]:
%%time
taxi.describe().toPandas()

CPU times: user 26.8 ms, sys: 3.86 ms, total: 30.7 ms
Wall time: 1.69 s


,summary,vendor_id,journey_length,year,month,dayofweek,pu_hour,do_hour,passenger_count,trip_distance,...,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,total_amount,pu_m_center,do_m_center,pu_m_jfk,do_m_jfk
0,count,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
1,mean,2.0,3.156,2011.566,6.458,4.211,0.0,0.058,1.68,0.6446600000000006,...,39.58942331989201,0.0,-71.93108716965665,39.46755911563793,1.0,5.00825,223812.1472677903,249762.91950719836,223812.1472677903,249762.91950719836
2,stddev,NaN,6.449593883436482,1.7597558753251912,3.475674943995238,2.1769745561715284,0.0,0.24636699595663514,1.2669669313779282,0.6041531970332402,...,8.451977333500384,0.0,12.701428196267504,8.724912232857061,NaN,0.6937944856283362,1461013.4557883241,1532171.4404206683,1461013.4557883241,1532171.4404206683
3,min,2,-4,2009,1,1,0,0,0,0.0,...,-74.005127,,-74.167993,-74.006007,1,2.5,858.6397937452996,859.2642971053626,858.6397937452996,859.2642971053626
4,max,VTS,178,2016,12,7,0,3,6,10.8,...,41.366138,Y,40.751217,41.366138,No Charge,5.5,1.5078256032059642E7,1.5078524665365847E7,1.5078256032059642E7,1.5078524665365847E7


Max total amount is fairly low $5.5. Have checked the full dataset and this is due to odd sampling. Shouldn't be too much of an issue for trial purposes.

In [10]:
# Check null values
taxi.select([count(when(isnan(c), c)).alias(c) for c in taxi.columns]).toPandas()

,vendor_id,journey_length,year,month,dayofweek,pu_hour,do_hour,passenger_count,trip_distance,pickup_longitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,total_amount,pu_m_center,do_m_center,pu_m_jfk,do_m_jfk
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
model_data = taxi.select("total_amount", "journey_length", "year", "month", 
                         "dayofweek", "pu_hour", "do_hour", 
                         "passenger_count", "trip_distance", 
                         "pu_m_center", 
                         "do_m_center", "pu_m_jfk", "do_m_jfk")

model_data.limit(5).toPandas()

,total_amount,journey_length,year,month,dayofweek,pu_hour,do_hour,passenger_count,trip_distance,pu_m_center,do_m_center,pu_m_jfk,do_m_jfk
0,2.5,0,2009,9,1,0,0,1,0.0,6340.676613,6320.606246,6340.676613,6320.606246
1,3.5,1,2009,11,1,0,0,1,0.0,5714.454928,5718.001754,5714.454928,5718.001754
2,5.5,1,2009,11,1,0,0,1,0.0,5403.354587,5403.354587,5403.354587,5403.354587
3,2.5,0,2009,7,7,0,0,1,0.0,4858.932353,4858.647049,4858.932353,4858.647049
4,3.5,0,2009,11,5,0,0,1,0.0,5515.768986,5518.134853,5515.768986,5518.134853


# Format data

MLlib requires data to be formated as two columns: label and features where features is a vector of all X values.

In [31]:
# Format data as label and features

# Define the `input_data` 
model_data_map = model_data.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# # Replace `df` with the new DataFrame
df = spark.createDataFrame(model_data_map, ["label", "features"])

df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  2.5|[0.0,2009.0,9.0,1...|
|  3.5|[1.0,2009.0,11.0,...|
|  5.5|[1.0,2009.0,11.0,...|
|  2.5|[0.0,2009.0,7.0,7...|
|  3.5|[0.0,2009.0,11.0,...|
|  3.0|[-4.0,2009.0,3.0,...|
|  2.5|[0.0,2009.0,7.0,6...|
|  2.5|[0.0,2009.0,7.0,4...|
|  3.0|[0.0,2009.0,5.0,2...|
|  3.0|[0.0,2009.0,5.0,6...|
|  2.5|[0.0,2009.0,8.0,7...|
|  3.5|[0.0,2009.0,12.0,...|
|  3.0|[0.0,2009.0,4.0,7...|
|  2.5|[0.0,2009.0,10.0,...|
|  3.0|[0.0,2009.0,3.0,4...|
|  3.0|[0.0,2009.0,3.0,7...|
|  3.0|[0.0,2009.0,4.0,3...|
|  2.5|[178.0,2009.0,7.0...|
|  4.5|[5.0,2009.0,6.0,6...|
|  5.5|[4.0,2009.0,12.0,...|
+-----+--------------------+
only showing top 20 rows



In [32]:
# Import `StandardScaler` 
from pyspark.ml.feature import StandardScaler

# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit the DataFrame to the scaler
scaler = standardScaler.fit(df)

# Transform the data in `df` with the scaler
scaled_df = scaler.transform(df)

# Inspect the result
scaled_df.take(2)

[Row(label=2.5, features=DenseVector([0.0, 2009.0, 9.0, 1.0, 0.0, 0.0, 1.0, 0.0, 6340.6766, 6320.6062, 6340.6766, 6320.6062]), features_scaled=DenseVector([0.0, 1141.6356, 2.5894, 0.4594, 0.0, 0.0, 0.7893, 0.0, 0.0043, 0.0041, 0.0043, 0.0041])),
 Row(label=3.5, features=DenseVector([1.0, 2009.0, 11.0, 1.0, 0.0, 0.0, 1.0, 0.0, 5714.4549, 5718.0018, 5714.4549, 5718.0018]), features_scaled=DenseVector([0.155, 1141.6356, 3.1649, 0.4594, 0.0, 0.0, 0.7893, 0.0, 0.0039, 0.0037, 0.0039, 0.0037]))]

# Split test and train

In [33]:
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

# Model data

In [34]:
rf = RandomForestRegressor(featuresCol="features_scaled")

In [35]:
model = rf.fit(train_data)

In [17]:
predictions = model.transform(test_data)

In [18]:
predictions.show()

+-----+--------------------+--------------------+------------------+
|label|            features|     features_scaled|        prediction|
+-----+--------------------+--------------------+------------------+
|  2.5|[0.0,2009.0,7.0,4...|[0.0,1141.6356258...| 2.987551608906712|
|  2.5|[0.0,2009.0,7.0,6...|[0.0,1141.6356258...|2.9884108373975193|
|  2.5|[0.0,2009.0,7.0,7...|[0.0,1141.6356258...|3.1153045686662506|
|  3.0|[0.0,2009.0,4.0,7...|[0.0,1141.6356258...| 3.070418205029887|
|  3.5|[0.0,2010.0,2.0,5...|[0.0,1142.2038864...|3.6400573752811822|
|  3.5|[0.0,2010.0,4.0,4...|[0.0,1142.2038864...| 3.661391474375205|
|  3.5|[0.0,2010.0,9.0,6...|[0.0,1142.2038864...|3.5992484188839287|
|  3.5|[0.0,2011.0,1.0,7...|[0.0,1142.7721470...| 4.149842583050291|
|  3.5|[0.0,2012.0,1.0,4...|[0.0,1143.3404077...| 3.791684729769108|
|  3.5|[0.0,2012.0,4.0,1...|[0.0,1143.3404077...|3.7201844627579232|
|  3.5|[0.0,2012.0,4.0,3...|[0.0,1143.3404077...| 3.608896418992745|
|  3.5|[0.0,2013.0,12.0,...|[0.0,1

# Evaluate data

In [21]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = {:.2f}".format(rmse))

Root Mean Squared Error (RMSE) on test data = 0.29


In [22]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = {:.2f}".format(r2))

Root Mean Squared Error (RMSE) on test data = 0.84


In [23]:
spark.catalog.clearCache()

# 100K rows

In [24]:
# Load data from BigQuery.
taxi= spark.read.format('bigquery').option('table', 'sap-ds-demo:big_data_demo_ew2.nyc_taxi_100000').load()
taxi.createOrReplaceTempView('taxi')

In [25]:
taxi.count()

100000

In [27]:
def prep_model_data(df):
    
    print("Preparing model data")
    
    model_data = df.select("total_amount", "journey_length", "year", "month", 
                         "dayofweek", "pu_hour", "do_hour", 
                         "passenger_count", "trip_distance", 
                         "pu_m_center", 
                         "do_m_center", "pu_m_jfk", "do_m_jfk")
    
    model_data_map = model_data.rdd.map(lambda x: (x[0], DenseVector(x[1:])))    
    train_test = spark.createDataFrame(model_data_map, ["label", "features"])
    train, test = train_test.randomSplit([.8,.2],seed=1234)
    return train, test

def model_data(train, test):
    
    print("Modelling data")
    
    rf = RandomForestRegressor(featuresCol="features")
    model = rf.fit(train_data)
    predictions = model.transform(test_data)
    
    print("Evaluating test predictions")
    
    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    
    evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
    r2 = evaluator.evaluate(predictions)
    
    print("RMSE: {:.2f}, R2 {:.2f}".format(rmse, r2))

In [28]:
train, test = prep_model_data(taxi)

print("Train count: {}. Test count: {}".format(train.count(), test.count()))

Preparing model data
Train count: 79977. Test count: 20023


In [29]:
model_data(train, test)

Modelling data
Evaluating test predictions
RMSE: 0.29, R2 0.84


In [30]:
spark.catalog.clearCache()

# 1 million rows

In [35]:
# Load data from BigQuery.
taxi= spark.read.format('bigquery').option('table', 'sap-ds-demo:big_data_demo_ew2.nyc_taxi_1m').load()
taxi.createOrReplaceTempView('taxi')

In [36]:
taxi.count()

100000000

In [37]:
train, test = prep_model_data(taxi)

Preparing model data


In [ ]:
print("Train count: {}. Test count: {}".format(train.count(), test.count()))

In [ ]:
model_data(train, test)

In [ ]:
spark.catalog.clearCache()

# 1 billion rows

In [ ]:
# Load data from BigQuery.
taxi= spark.read.format('bigquery').option('table', 'sap-ds-demo:big_data_demo_ew2.nyc_taxi_model_input').load()
taxi.createOrReplaceTempView('taxi')

In [ ]:
taxi.count()

In [ ]:
train, test = prep_model_data(taxi)


In [ ]:
train.count()